AVANCE 1

1) Hago IMPORT de las bibliotecas numpy & pandas

In [ ]:
import numpy as np
import pandas as pd

2) Creo una variable para el archivo csv 

In [ ]:
archivo_data_latinoamerica = 'data_latinoamerica.csv'

3) Genero otra variable DateFrame donde leo la variable antes creada que tiene el archivo.

In [ ]:
df_data_latinoamerica = pd.read_csv(archivo_data_latinoamerica, parse_dates=['date'])

4) Compruebo la cantidad de registros y columnas usando .info() | .isnull().sum()

In [ ]:
df_data_latinoamerica.info()
df_data_latinoamerica.isnull().sum()

5) Selecciono los paises a los cuales se expandiran, que son: Colombia, Argentina, Chile, Mexico, Peru y Brazil.

In [ ]:
paises_filtrados = ['Colombia','Argentina','Chile','Mexico','Peru','Brazil']
df_data_latinoamerica_expansion = df_data_latinoamerica[df_data_latinoamerica['country_name'].isin(paises_filtrados)]

Filtro datos a partir de location_key

In [ ]:
df_data_latinoamerica_expansion_clean = df_data_latinoamerica_expansion[df_data_latinoamerica_expansion['location_key'].isin(['AR','CL','CO','MX','PE','BR'])]

In [ ]:
df_data_latinoamerica_expansion_clean.info()

In [ ]:
df_data_latinoamerica_expansion_clean.sample(20)

In [ ]:
df_data_latinoamerica_expansion_clean.isnull().sum()

Reemplazo nulos con la media de cada columna correspondiente y elimino la columna 'country_code' ya que tiene los mismos valores que 'location_key'

In [ ]:
df_data_filled = df_data_latinoamerica_expansion_clean.apply(lambda col: col.fillna(0).fillna(col.shift(1) + 1) if col.dtype in ['int64', 'float64'] else col, axis=0)
df_data_filled = df_data_filled.drop(columns=['country_code'])

In [ ]:
df_data_filled.sample(15)

In [ ]:
df_data_filled.isnull().sum()

6) Filtrar datos a partir del 2021-01-01

In [ ]:
df_expansion = df_data_filled[(df_data_filled['date'] >= '2021-01-01')]

In [ ]:
df_expansion.info() # Con los filtros consegui reducir el numero de entries a 3750 en 49 columnas.

In [ ]:
df_expansion.isnull().sum()

8) Crear un archivo llamado 'DatosFinalesFiltrado.csv' para trabajar sobre este archivo.

In [ ]:
df_expansion.to_csv('DatosFinalesFiltrado.csv', index=False)

========================================================================================================================================

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
datos_finales = 'DatosFinalesFiltrado.csv'
df_datos = pd.read_csv(datos_finales)

In [ ]:
df_datos['date'] = pd.to_datetime(df_datos['date']) #Paso la columna date a type datetime.

In [ ]:
df_datos.info() #3750 entries, 49 columns, 1 datetime, 46 float, 2 object.

In [ ]:
df_datos.sample(3)

9) Aplica bucles for y/o while para el cálculo de estadísticas descriptivas y otras métricas importantes que ofrece pandas por default. Luego responde:

¿Qué implican estas métricas y cómo pueden ayudar en el análisis de datos?

¿Se muestran todas las estadísticas en todas las columnas durante el análisis?

¿Cuál es la razón de la respuesta anterior y cómo podría afectar la interpretación de los resultados obtenidos?

In [ ]:
# funcion para mostrar el pais con mas tasa de mortalidad en mujeres y hombres, mostrar pais con mas muertes en general
def mortalidad(df, pais, mujeres, hombres, muertes_totales):
        x_mujeres = df[mujeres].idxmax()
        x_hombres = df[hombres].idxmax()
        x_total = df[muertes_totales].idxmax()

        tasa_mujeres = df.loc[x_mujeres, pais]
        tasa_hombres = df.loc[x_hombres, pais]
        tasa_total = df.loc[x_total, pais]

        return {'Pais con mayor tasa de mortalidad en mujeres:':tasa_mujeres,
                'Pais con mayor tasa de mortalidad en hombres:':tasa_hombres,
                'Pais con mayor tasa de mortalidad en general:':tasa_total
        }


In [ ]:
resultado = mortalidad(df_datos, 'country_name', 'adult_female_mortality_rate', 'adult_male_mortality_rate', 'cumulative_deceased')
print(resultado)

In [ ]:
# mostrar pais con mas casos acumulados y el que menos casos tiene
def casos(df, pais, casos):
    x_casos_max = df[casos].idxmax()
    x_casos_min = df[casos].idxmin()
    
    mas_casos = df.loc[x_casos_max, pais]
    menos_casos = df.loc[x_casos_min, pais]
    
    return {'Pais con mas casos acumulados:': mas_casos,
            'Pais con menos casos acumulados:': menos_casos
            }

In [ ]:
resultado2 = casos(df_datos, 'country_name', 'cumulative_confirmed')
print(resultado2)

10) Crea una función que permita obtener la mediana, varianza y el rango. Te dejamos algunas preguntas para analizar esas medidas:

¿Qué representa la mediana?

¿Cómo varía la dispersión de los datos en el conjunto de datos analizado, en términos de la varianza y el rango? 

¿Qué nos puede indicar esto sobre la consistencia o la variabilidad de los datos en relación con la mediana?

In [ ]:
def medidas(df, columna):
    mediana = df[columna].median()
    varianza = df[columna].var()
    rango = df[columna].max() - df[columna].min()
    return {'Mediana': mediana, 'Varianza': varianza, 'Rango': rango}

In [ ]:
while True:
    try:
        opcion = int(input('Que columna desea calcular:\n1-Casos nuevos confirmados\n2-Muertes nuevas confirmadas\n3-Dosis dadas'))
        match opcion:
            case 1:
                print(f'Medidas de casos confirmados:\n{medidas(df_datos,'new_confirmed')}')
            case 2:
                print(f'Medidas de muertes confirmadas:\n{medidas(df_datos,'new_deceased')}')
            case 3:
                print(f'Medidas de vacunas administradas:\n{medidas(df_datos, 'cumulative_vaccine_doses_administered' )}')
            case 4:
                print('Cerrando...')
                break
            case _:
                print('Ingrese un numero del 1 al 4')
                continue
    except ValueError:
        print('Ingrese un numero valido.')
        continue

==========================================================================================================================================

AVANCE 2

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
datos_finales = 'DatosFinalesFiltrado.csv'
df_datos = pd.read_csv(datos_finales)

In [ ]:
df_datos['date'] = pd.to_datetime(df_datos['date'])

In [ ]:
df_datos.info()

In [ ]:
df_datos.sample(5)

In [ ]:
df_datos.columns

In [ ]:
for i in df_datos.columns:
    print(i)
    print(df_datos[i].describe())
    print('---------------------------------')

In [ ]:
# Histogramas gráficos de densidad y gráficos de densidad para entender la distribución de la incidencia de COVID-19 y las tasas de vacunación.
plt.figure(figsize=(10,5))
sns.histplot(
    data=df_datos,
    x='cumulative_vaccine_doses_administered',
    kde=True
)
plt.title('Distribucion de dosis administradas.')
plt.xlabel('Casos confirmados')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Histograma para casos confirmados de COVID-19
plt.figure(figsize=(10,5))
sns.histplot(
    data=df_datos,
    x='cumulative_confirmed',
    kde=True
    )
plt.title('Distribucion de casos confirmados de COVID-19')
plt.xlabel('Casos confirmados')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df_datos.columns

In [ ]:
df_datos.sample(5)

In [ ]:
df_datos['menores'] = df_datos[['population_age_00_09','population_age_10_19']].sum(axis=1)

fig, ax = plt.subplots(figsize=(15, 5))
sns.barplot(
    x='country_name',
    y='menores',
    data=df_datos,
    hue='country_name'
)
plt.title('Casos totales de menores confirmados por pais')
plt.ylabel('Casos')
plt.xlabel('Pais')
plt.show()

In [ ]:
df_datos['mayores'] = df_datos[
    ['population_age_20_29','population_age_30_39','population_age_40_49','population_age_50_59',
    'population_age_60_69','population_age_70_79','population_age_80_and_older']].sum(axis=1)

fig, ax = plt.subplots(figsize=(15, 5))
sns.barplot(
    x='country_name',
    y='mayores',
    data=df_datos,
    hue='country_name'
)
plt.title('Casos totales de menores confirmados por pais')
plt.ylabel('Casos')
plt.xlabel('Pais')
plt.show()

In [ ]:
# Gráficos de barras para comparar diferentes regiones.
for i in df_datos:
    if i not in ['location_key', 'date', 'country_code', 'country_name', 'latitude', 'longitude', 'cumulative_confirmed', 'cumulative_deceased', 'cumulative_recovered',
                'cumulative_vaccine_doses_administered', 'new_recovered']:
        fig, ax = plt.subplots(figsize=(5, 3))
        sns.barplot(
            x='country_name',
            y=i,
            data=df_datos,
            ax=ax,
            hue='country_name'
        )
        plt.title(f'Columna: {i} \nPor pais en latinoamerica.')
        plt.xlabel('Pais')
        plt.show()

In [ ]:
# Diagrama de dispersión de la temperatura media contra los casos confirmados.
fig, ax = plt.subplots(figsize=(10,6))
sns.scatterplot(
    x='average_temperature_celsius',
    y='cumulative_confirmed',
    data=df_datos,
    hue='country_name'
)
plt.xlabel('Temperaturas')
plt.ylabel('Casos Confirmadas')
plt.title('Promedio de las temperaturas vs casos confirmados.')

plt.show()

In [ ]:
# Diagrama de dispersión de la temperatura media contra las muertes confirmadas.
fig, ax = plt.subplots(figsize=(10,6))
sns.scatterplot(
    x='average_temperature_celsius',
    y='cumulative_deceased',
    data=df_datos,
    hue='country_name'
)
plt.xlabel('Temperaturas')
plt.ylabel('Muertes Confirmadas')
plt.title('Promedio de las temperaturas vs muertes confirmadas.')

plt.show()

In [ ]:
# Matriz de correlación, trata de crear una submascara para solo tomar una parte del gráfico y filtrar por los valores mayores a 0.5.
numeric_columns = df_datos.select_dtypes(include=['float64', 'int64']).columns
correlacion_matrix = df_datos[numeric_columns].corr() # Creo la matriz de correlacion 

mask = np.triu(np.ones_like(correlacion_matrix, dtype=bool)) # Creo una mascara

correlacion_matrix = correlacion_matrix[abs(correlacion_matrix) > abs(0.5)] # Muestro valores mayores a 0.5

plt.figure(figsize=(25, 10))
heatmap = sns.heatmap(
    correlacion_matrix,
    mask=mask,
    annot=True,
    cmap='coolwarm',
    fmt='.2f'
    )
plt.title('Matriz de correlacion de las variables numericas.', fontsize=20)
plt.show()

In [ ]:
df_datos.sample(3)

In [ ]:
df_datos.columns

In [ ]:
# Evolución de dosis administradas por mes de cada país.

# Creo una nueva columna y extraigo el mes de cada fecha.
df_datos['mes'] = df_datos['date'].dt.to_period('M')
df_datos = df_datos.sort_values(['country_name', 'date'])

# Creo una columna llamada 'dosis_mes' donde agrupo por pais calculando cada fila de la columna de dosis adm con .diff() y rellenando las filas que hayan podido quedar nulas con .fillna(0)
df_datos['dosis_mes'] = df_datos.groupby('country_name')['cumulative_vaccine_doses_administered'].diff().fillna(0)
# Limito la columna para que no hayan valores menores a 0 que rompa el grafico.
df_datos['dosis_mes'] = df_datos['dosis_mes'].clip(lower=0)
# Agrupo en una nueva variable por mes y pais, y sumo la columna de 'dosis_mes' 
df_dosis_mes = df_datos.groupby(['mes', 'country_name'])['dosis_mes'].sum().reset_index()
# Me aseguro que el tipo de la columna 'mes' sea str
df_dosis_mes['mes'] = df_dosis_mes['mes'].astype(str) 

# Grafico
plt.subplots(figsize=(12, 8))
sns.barplot(
    x='mes',
    y='dosis_mes',
    hue='country_name',
    data=df_dosis_mes 
)
plt.title('Evolucion de dosis administradas por mes en cada pais.')
plt.xlabel('Mes')
plt.ylabel('Dosis administradas')
plt.xticks(rotation=45)
plt.legend(title='Pais', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Muertes por mes de cada país.

# Creo una nueva columna y extraigo el mes de cada fecha.
df_datos['mes'] = df_datos['date'].dt.to_period('M')
df_datos = df_datos.sort_values(['country_name', 'date'])

# Agrupo en una nueva variable por mes y pais, y sumo la columna de 'new_deceased' 
df_muertes_mes = df_datos.groupby(['mes', 'country_name'])['new_deceased'].sum().reset_index()
# Sorteo los valores por mes.
df_muertes_mes = df_muertes_mes.sort_values(by='mes')
# Me aseguro que el tipo de la columna 'mes' sea str
df_muertes_mes['mes'] = df_muertes_mes['mes'].astype(str)

plt.subplots(figsize=(12, 8))
# Creo el grafico de barras
sns.barplot(
    x='mes',
    y='new_deceased',
    hue='country_name',
    data=df_muertes_mes
)
plt.title('Evolucion de muertes por mes en cada pais.')
plt.xlabel('Mes')
plt.ylabel('Muertes')
plt.xticks(rotation=45)
plt.legend(title='Pais', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Casos confirmados por mes de cada país.

# Creo una nueva columna y extraigo el mes de cada fecha.
df_datos['mes'] = df_datos['date'].dt.to_period('M')
# Agrupo en una nueva variable por mes y pais, y sumo la columna de 'new_confirmed' 
df_casos_mes = df_datos.groupby(['mes', 'country_name'])['new_confirmed'].sum().reset_index()
# Me aseguro que el tipo de la columna 'mes' sea str
df_casos_mes['mes'] = df_casos_mes['mes'].astype(str)
# Sorteo los valores por mes.
df_casos_mes = df_casos_mes.sort_values(by='mes')

plt.subplots(figsize=(12, 8))
# Creo el grafico de barras
sns.barplot(
    x='mes',
    y='new_confirmed',
    hue='country_name',
    data=df_casos_mes # Uso la variable que cree antes
)
plt.title('Evolucion de casos confirmados por mes en cada pais.')
plt.xlabel('Mes')
plt.ylabel('Casos')
plt.xticks(rotation=45)
plt.legend(title='Pais', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Recuperaciones de casos por mes de cada país.


# Creo una nueva columna y extraigo el mes de cada fecha.
df_datos['mes'] = df_datos['date'].dt.to_period('M')
# Agrupo en una nueva variable por mes y pais, y sumo la columna de 'new_recovered' 
df_recuperados_mes = df_datos.groupby(['mes', 'country_name'])['new_recovered'].sum().reset_index()
# Sorteo los valores por mes.
df_recuperados_mes = df_recuperados_mes.sort_values(by='mes')
# Me aseguro que el tipo de la columna 'mes' sea str
df_recuperados_mes['mes'] = df_recuperados_mes['mes'].astype(str)

plt.subplots(figsize=(12, 8)) # Creo la figura
# Creo el grafico de barras
sns.barplot(
    x='mes',
    y='new_recovered',
    hue='country_name',
    data=df_recuperados_mes # Uso la variable que cree antes
)
plt.title('Evolucion de casos recuperados por mes en cada pais.')
plt.xlabel('Mes')
plt.ylabel('Recuperaciones')
plt.xticks(rotation=45)
plt.legend(title='Pais', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Comparación del Número de Casos Nuevos entre Países.

plt.subplots(figsize=(12, 8))
# Creo el grafico de barras
sns.barplot(
    x='mes',
    y='new_confirmed',
    hue='country_name',
    data=df_datos,
)
plt.title('Evolucion de nuevos casos en cada pais.')
plt.xlabel('Mes')
plt.ylabel('Nuevos casos')
plt.xticks(rotation=45)
plt.legend(title='Pais', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Boxplot de temperatura media de cada país.

plt.subplots(figsize=(20,10))
sns.boxplot(
    data=df_datos,
    x='mes',
    y='average_temperature_celsius',
    hue='country_name'
)
plt.title('Temperatura media de cada pais por mes.')
plt.xlabel('Mes')
plt.ylabel('Temperatura')
plt.xticks(rotation=45)
plt.show()

==========================================================================================================================================

AVANCE 3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [ ]:
# Codigo para ignorar warnings 'molestos'.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Codigo pandas para mostrar solo 1 decimal luego del punto.
pd.options.display.float_format = '{:.1f}'.format

In [ ]:
datos_finales = 'DatosFinalesFiltrado.csv'
df_datos = pd.read_csv(datos_finales)
df_datos['date'] = pd.to_datetime(df_datos['date'])

In [ ]:
# Este codigo lo uso mas adelante en el grafico de Life Expectancy.
path = 'ne_110m_admin_0_countries.zip'
world = gpd.read_file(path)
df_datos_merged = world.merge(df_datos, left_on='SOVEREIGNT', right_on='country_name')

In [ ]:
# df_datos_indexdate = df_datos.set_index('date')
# df_datos_indexdate.info()
# df_datos_indexdate.head(4)

In [ ]:
df_datos.info()

In [ ]:
df_datos.describe()

In [ ]:
df_datos.sample(4)

In [ ]:
df_datos.isnull().sum()

In [ ]:
df_datos.columns

In [ ]:
# Casos confirmados y casos recuperados por year
df_datos['Year'] = df_datos['date'].dt.year
suma_anual = df_datos.groupby('Year')[['cumulative_confirmed','cumulative_recovered']].sum()
suma_anual = suma_anual.reset_index('Year')
suma_anual

In [ ]:
# Relación entre la Cobertura de Vacunación y la Reducción de Casos
df_datos['mes'] = df_datos['date'].dt.to_period('M')

plt.figure(figsize=(15,10))
sns.scatterplot(
    data=df_datos,
    x='cumulative_vaccine_doses_administered',
    y='new_recovered',
    hue='mes',
)
plt.title('Efectividad de las dosis administradas y recuperados.')
plt.xlabel('Dosis Administradas')
plt.ylabel('Nuevos Recuperados')
plt.legend(title='Fecha', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
df_datos['mes'] = df_datos['date'].dt.to_period('M')

# Relación entre la Cobertura de Vacunación y muertes
plt.figure(figsize=(15,10))
sns.scatterplot(
    data=df_datos,
    x='cumulative_vaccine_doses_administered',
    y='new_deceased',
    hue='mes',
    #hue='country_name',
)
plt.title('Dosis administradas y fallecidos.')
plt.xlabel('Dosis Administradas')
plt.ylabel('Muertes')
plt.legend(title='Fecha', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Nuevos casos y temperatura promedio
plt.figure(figsize=(15,6))
sns.scatterplot(
    data=df_datos,
    x='average_temperature_celsius',
    y='new_confirmed',
    hue='mes',
    style='mes'
)
plt.xlabel('Temperatura')
plt.ylabel('Nuevos casos')
plt.legend(title='Fecha', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.show()

In [ ]:
# Variación Mensual de Casos y Muertes
df_datos['mes'] = df_datos['mes'].astype(str)

#Calculo la variacion en nuevas columnas
df_datos['var_casos'] = df_datos['new_confirmed'].diff()
df_datos['var_muertes'] = df_datos['cumulative_deceased'].diff()

plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df_datos,
    x="mes", 
    y="var_casos", 
    marker="o", 
    ci=None,
    label="Variación Casos", 
    color="blue"
    )
sns.lineplot(
    data=df_datos,
    x="mes", 
    y="var_muertes", 
    marker="o", 
    ci=None,
    label="Variación Muertes", 
    color="red"
    )

plt.xlabel("Mes y Año")
plt.ylabel("Variación")
plt.title("Variación Mensual de Casos y Muertes")
plt.legend()
plt.xticks(rotation=45) 
plt.tight_layout()
plt.show()

In [ ]:
# EXTRA CREDIT
import geopandas as gpd

path = 'ne_110m_admin_0_countries.zip'
world = gpd.read_file(path)
df_datos_merged = world.merge(df_datos, left_on='SOVEREIGNT', right_on='country_name')
df_datos_merged.plot(
    column='life_expectancy',
    figsize=(15,10),
    legend=True,
    edgecolor='black',
    
    legend_kwds={
        'label': 'Expectativa',
        'orientation': 'horizontal'
    })
plt.title('Expectativa de vida por Pais')
plt.axis('off')
plt.show()

==================================================================================================================================

AVANCE 4

In [ ]:
! pip install mysql-connector-python
! pip install pymssql
! pip install pyodbc

In [ ]:
# Codigo para importar el df de 'DatosFinalesFiltrado.csv' en Power BI
import pandas as pd
datos_finales = 'D:/Data Analitycs/M4/VSC/Proyecto/DatosFinalesFiltrado.csv'
dataset = pd.read_csv(datos_finales)

print(dataset) 

In [ ]:
# Codigo que implemento al principio de cada script en Power Bi
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

df_datos = pd.read_csv(r'D:/Data Analitycs/M4/VSC/Proyecto/DatosFinalesFiltrado.csv')
df_datos['date'] = pd.to_datetime(df_datos['date'])